<a href="https://colab.research.google.com/github/lunecarvalho/categorize-product/blob/main/categorize_product.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install transformers==4.40.2

In [17]:
from transformers import pipeline

In [18]:
classificador = pipeline('zero-shot-classification', model='facebook/bart-large-mnli')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:949: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [19]:
categorias = ['eletronics','food','toys','books']

In [20]:
predicao = classificador('''
Latest model of smartphone with 5G connectivity and 128GB internal storage''', candidate_labels=categorias)

In [21]:
predicao

{'sequence': '\nLatest model of smartphone with 5G connectivity and 128GB internal storage',
 'labels': ['eletronics', 'food', 'books', 'toys'],
 'scores': [0.9009801149368286,
  0.03743201866745949,
  0.032767001539468765,
  0.028820887207984924]}

In [22]:
!pip install liqfit sentencepiece

In [23]:
from liqfit.pipeline import ZeroShotClassificationPipeline
from liqfit.models import T5ForZeroShotClassification
from transformers import T5Tokenizer

model = T5ForZeroShotClassification.from_pretrained('knowledgator/comprehend_it-multilingual-t5-base')
tokenizer = T5Tokenizer.from_pretrained('knowledgator/comprehend_it-multilingual-t5-base')
classifier = ZeroShotClassificationPipeline(model=model, tokenizer=tokenizer,
                                                      hypothesis_template = '{}', encoder_decoder = True)


You are using a model of type T5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


In [24]:
descricao = '''Este water gel leve e refrescante, proporciona hidratação imediata que ajuda a aliviar o repuxamento e aspereza da pele sensível'''

categorias_candidatas = ['beleza','cozinha','livros']

resultado = classifier(descricao, categorias_candidatas, multi_label=False)

resultado

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'sequence': 'Este water gel leve e refrescante, proporciona hidratação imediata que ajuda a aliviar o repuxamento e aspereza da pele sensível',
 'labels': ['beleza', 'livros', 'cozinha'],
 'scores': [0.9420967698097229, 0.03873441740870476, 0.019168810918927193]}

In [25]:
import pandas as pd

In [26]:
resultado = pd.DataFrame(resultado).drop(['sequence'], axis=1)
resultado

,labels,scores
0,beleza,0.942097
1,livros,0.038734
2,cozinha,0.019169


In [27]:
dados = pd.read_csv('https://raw.githubusercontent.com/alura-cursos/hugging_face/refs/heads/main/Dados/descricoes_produtos.csv')
dados

,Descrição
0,Liquidificador de alta potência com jarra de v...
1,"Forno Micro-ondas de 20 litros, com menu desco..."
2,Máquina de café espresso com reservatório de á...
3,Torradeira com capacidade para quatro fatias e...
4,"Panela elétrica multifuncional que cozinha, as..."
5,Smartphone com 128GB de armazenamento e câmera...
6,"Smartwatch monitoramento de passos, frequência..."
7,Notebook gamer com placa de vídeo potente e te...
8,"Smartphone com tela super retina XDR de 6,7 po..."
9,Câmera digital com zoom óptico de 20x e vídeos...


In [29]:
categorias = ['eletrodomésticos', 'eletrônicos', 'beleza', 'brinquedos']

def categorizar(descricao):
  resultado = classifier(descricao, categorias, multi_label=False)
  categoria_max = max(zip(resultado['labels'], resultado['scores']), key=lambda x: x[1])[0]
  return categoria_max

dados['Categoria'] = dados['Descrição'].apply(categorizar)

In [30]:
dados

,Descrição,Categoria
0,Liquidificador de alta potência com jarra de v...,eletrodomésticos
1,"Forno Micro-ondas de 20 litros, com menu desco...",eletrodomésticos
2,Máquina de café espresso com reservatório de á...,eletrodomésticos
3,Torradeira com capacidade para quatro fatias e...,eletrodomésticos
4,"Panela elétrica multifuncional que cozinha, as...",eletrodomésticos
5,Smartphone com 128GB de armazenamento e câmera...,eletrônicos
6,"Smartwatch monitoramento de passos, frequência...",eletrônicos
7,Notebook gamer com placa de vídeo potente e te...,eletrônicos
8,"Smartphone com tela super retina XDR de 6,7 po...",eletrônicos
9,Câmera digital com zoom óptico de 20x e vídeos...,eletrônicos
